<a href="https://colab.research.google.com/github/prawizard/TweetsClassification_NLP/blob/main/Sentiment_Analysis_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RNNs

We will use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras.  Conveniently, Keras has a built-in IMDb movie reviews dataset that we can use.

In [1]:
from keras.datasets import imdb

# A

In [104]:
vocabulary_size = 17110

# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
# print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

 Inspect a sample review and its label

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
import re
import requests

In [24]:
TRAIN_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt"
TRAIN_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt"
VAL_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt"
VAL_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt"
TEST_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt"
TEST_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt"

In [25]:
r = requests.get(TRAIN_TEXT_URL, allow_redirects=True)
open('train_text.txt', 'wb').write(r.content)

r = requests.get(TRAIN_LABELS_URL, allow_redirects=True)
open('train_labels.txt', 'wb').write(r.content)

r = requests.get(VAL_TEXT_URL, allow_redirects=True)
open('val_text.txt', 'wb').write(r.content)

r = requests.get(VAL_LABELS_URL, allow_redirects=True)
open('val_labels.txt', 'wb').write(r.content)

r = requests.get(TEST_TEXT_URL, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r = requests.get(TEST_LABELS_URL, allow_redirects=True)
open('test_labels.txt', 'wb').write(r.content)

1720

In [26]:
stream=open("train_text.txt")
tweets=stream.readlines()
stream.close()

val_stream=open("val_text.txt")
val_tweets=val_stream.readlines()
val_stream.close()

test_stream=open("test_text.txt")
test_tweets=test_stream.readlines()
test_stream.close()

In [27]:
for i in range(len(tweets)):
  if tweets[i].find('@user')!=-1:
    # tweets[i] = tweets[i].replace('@user',"")
    tweets[i]=re.sub('@user', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('@user')!=-1:
    val_tweets[i]=re.sub('@user', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('@user')!=-1:
    test_tweets[i]=re.sub('@user', '', test_tweets[i])

In [28]:
stream=open("train_labels.txt")
tweetsLabels=stream.readlines()
stream.close()

val_stream=open("val_labels.txt")
val_tweetsLabels=val_stream.readlines()
val_stream.close()

test_stream=open("test_labels.txt")
test_tweetsLabels=test_stream.readlines()
test_stream.close()

In [29]:
labels=[0]*len(tweetsLabels)
for i in range(len(tweetsLabels)):
  if tweetsLabels[i].find('\n')!=-1:
    # tweets[i] = tweets[i].replace('@user',"")
    labels[i]=int(re.sub('\n', '', tweetsLabels[i]))
# labels    
# Words like effing converted to VetsResistSquadron

val_labels=[0]*len(val_tweetsLabels)
for i in range(len(val_tweetsLabels)):
  if val_tweetsLabels[i].find('\n')!=-1:
    # tweets[i] = tweets[i].replace('@user',"")
    val_labels[i]=int(re.sub('\n', '', val_tweetsLabels[i]))

test_labels=[0]*len(test_tweetsLabels)
for i in range(len(test_tweetsLabels)):
  if test_tweetsLabels[i].find('\n')!=-1:
    # tweets[i] = tweets[i].replace('@user',"")
    test_labels[i]=int(re.sub('\n', '', test_tweetsLabels[i]))

In [30]:
rows=[]
rowIndices=[]
for i in range(len(tweets)):
  rows.append({"TWEET":tweets[i], "CATEGORY":labels[i]})
  rowIndices.append(i+1)
df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
for i in range(len(val_tweets)):
  val_rows.append({"TWEET":val_tweets[i], "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
for i in range(len(test_tweets)):
  test_rows.append({"TWEET":test_tweets[i], "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [37]:
test_df.head()

,TWEET,CATEGORY
1,#ibelieveblaseyford is liar she is fat ugly li...,1
2,I got in a pretty deep debate with my frien...,0
3,"...if you want more shootings and more death, ...",0
4,Angels now have 6 runs. Five of them have come...,0
5,#Travel #Movies and Unix #Fortune combined Vi...,0


In [32]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [34]:
def clean_tweets(tweet, stemmer=PorterStemmer(), stop_words=set(stopwords.words('english'))):
  words=word_tokenize(tweet.lower())
  filtered_words=[]
  # for word in words:
  #   if word not in stop_words and word.isalpha():
  #     stemmed_word=stemmer.stem(word)
  #     filtered_words.append(stemmed_word)
  for word in words:
    if word.isalpha():
      # stemmed_word=stemmer.stem(word)
      filtered_words.append(word)
  return filtered_words

In [45]:
tweetsList=df.TWEET
nestedList=tweetsList.apply(clean_tweets)
# nestedList[0:5]
val_tweetsList=val_df.TWEET
val_nestedList=val_tweetsList.apply(clean_tweets)
# val_nestedList[0:5]
test_tweetsList=test_df.TWEET
test_nestedList=test_tweetsList.apply(clean_tweets)
test_nestedList[0:5]

1    [ibelieveblaseyford, is, liar, she, is, fat, u...
2    [i, got, in, a, pretty, deep, debate, with, my...
3    [if, you, want, more, shootings, and, more, de...
4    [angels, now, have, runs, five, of, them, have...
5    [travel, movies, and, unix, fortune, combined,...
Name: TWEET, dtype: object

In [46]:
nestedList=tuple(nestedList)
val_nestedList=tuple(val_nestedList)
test_nestedList=tuple(test_nestedList)

In [ ]:
test_nestedList[:5]

In [56]:
uniquewords_train=set()
for tw in nestedList:
  for w in tw:
    uniquewords_train.add(w)

In [59]:
uniquewords_train=tuple(uniquewords_train)
word_id={}
id_word={}
i=1
for word in uniquewords_train:
  word_id[word]=i
  i+=1

for word, id in word_id.items():
  id_word[id]=word


In [99]:
len(uniquewords_train)

17110

In [90]:
X_train=[]
twt=[]
for tw in nestedList:
  for word in tw:
    wordid=word_id[word]
    twt.append(wordid)
  X_train.append(twt)
  twt=[]

twt=[]
X_val=[]
for tw in val_nestedList:
  for word in tw:
    wordid=word_id.get(word, 0)
    twt.append(wordid)
  X_val.append(twt)
  twt=[]

twt=[]
X_test=[]
for tw in test_nestedList:
  for word in tw:
    wordid=word_id.get(word, 0)
    twt.append(wordid)
  X_test.append(twt)
  twt=[]


In [91]:
print(X_train[6])
print([id_word.get(i, ' ') for i in X_train[6]])
print()
print(X_test[6])
print([id_word.get(i, ' ') for i in X_test[6]])
print()
print(X_val[6])
print([id_word.get(i, ' ') for i in X_val[6]])

[6518, 10968, 7569, 1710, 1820, 2582, 11319, 2650]
['your', 'looking', 'more', 'like', 'a', 'plant', 'maga', 'walkaway']

[4770, 395, 799, 5225, 0]
['this', 'speaks', 'for', 'itself', ' ']

[6328, 13311, 12615, 5713, 6362, 6805, 16765, 16076, 12148, 5713, 4189, 10030, 2656, 799, 6044, 918]
['there', 'r', 'million', 'that', 'can', 'sign', 'to', 'the', 'affect', 'that', 'they', 'did', 'vote', 'for', 'an', 'asshole']


In [92]:
y_train=df.CATEGORY
y_val=val_df.CATEGORY
y_test=test_df.CATEGORY

# B

In [22]:
type(y_test)

numpy.ndarray

In [88]:
print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

---review---
[6518, 10968, 7569, 1710, 1820, 2582, 11319, 2650]
---label---
0


Map word IDs back to words

In [4]:
# word2id = imdb.get_word_index()
# id2word = {i: word for word, i in word2id.items()}
# print('---review with words---')
# print([id2word.get(i, ' ') for i in X_train[6]])
# print('---label---')
# print(y_train[6])

1646592/1641221 [==============================] - 0s 0us/step
---review with words---
['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'm

Maximum review length and minimum review length

In [93]:
print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

Maximum review length: 65


In [94]:
print('Minimum review length: {}'.format(
len(min((X_test + X_test), key=len))))

Minimum review length: 0


### Pad sequences

In order to feed this data into our RNN, all input documents must have the same length. We will limit the maximum review length to max_words by truncating longer reviews and padding shorter reviews with a null value (0). We can accomplish this using the pad_sequences() function in Keras. For now, set max_words to 500.

In [95]:
from keras.preprocessing import sequence

# max_words = 500
max_words = 65
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

### TODO: Design an RNN model for sentiment analysis

Build our model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that our input is a sequence of words (technically, integer word IDs) of maximum length = max_words, and our output is a binary sentiment label (0 or 1).

In [112]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 32)          547520    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 600,821
Trainable params: 600,821
Non-trainable params: 0
_________________________________________________________________
None


To summarize, our model is a simple RNN model with 1 embedding, 1 LSTM and 1 dense layers. 213,301 parameters in total need to be trained.

### Train and evaluate our model

We first need to compile our model by specifying the loss function and optimizer we want to use while training, as well as any evaluation metrics we'd like to measure. Specify the approprate parameters, including at least one metric 'accuracy'.

In [109]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

Once compiled, we can kick off the training process. There are two important training parameters that we have to specify - batch size and number of training epochs, which together with our model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a run!

In [110]:
batch_size = 64
num_epochs = 3

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

Epoch 1/3
  1/186 [..............................] - ETA: 7:44 - loss: 0.6926 - accuracy: 0.5625

InvalidArgumentError: ignored

scores[1] will correspond to accuracy if we pass metrics=['accuracy']

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.86964
